In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

In [3]:
base_url = 'https://www.brandi.co.kr/shop?tag=trend'

In [9]:
browser = Chrome()

In [10]:
browser.get(base_url)

In [16]:
browser.find_elements_by_xpath('//*[@class = "moreBtn"]/a')[0].click()

In [17]:
body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출

In [24]:
#스크롤 횟수는 11번으로 지정
num_of_pagedowns = 11
#10번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    num_of_pagedowns -= 1

In [27]:
browser.find_elements_by_xpath('//*[@class = "moreBtn"]/a')[0].click()

In [28]:
total_page = 10
num_of_pagedowns = 11

while total_page:
    total_page -=1
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_of_pagedowns -= 1   
    browser.find_elements_by_xpath('//*[@class = "moreBtn"]/a')[0].click()
    time.sleep(3)

IndexError: list index out of range

In [29]:
html = browser.page_source

In [30]:
html2 = BeautifulSoup(html,'html.parser')

In [34]:
body = html2.find('body')

In [35]:
app = body.find('div',{'id':'app'})

In [42]:
shop = app.find_all('div',{'class':'frame'})[5]

In [44]:
shop2 = shop.find('div',{'class':'list_store'})

In [45]:
len(shop2.find_all('ul'))

2227

In [46]:
shop3 = shop2.find_all('ul')

In [49]:
brandi  = 'https://www.brandi.co.kr'

In [56]:
shop3[2226].find('li',{'class':'list_store_tag'}).get_text()

'#20대후반#심플베이직#쇼핑몰'

In [58]:
brandi  = 'https://www.brandi.co.kr'
cscenter = '/cscenter'
brandi_shop = pd.DataFrame({'cs_url':[],
                           'shop_name':[],
                           'shop_tag':[]})

for i in range(len(shop3)):
    cs_url = brandi + shop3[i].find('li',{'class':'list_store_title'}).find('a')['href'] + '/cscenter'
    shop_name = shop3[i].find('li',{'class':'list_store_title'}).text
    shop_tag = shop3[i].find('li',{'class':'list_store_tag'}).get_text()
    
    insert_data = pd.DataFrame({'cs_url':[cs_url],
                                'shop_name':[shop_name],
                               'shop_tag':[shop_tag]})
    
    brandi_shop = brandi_shop.append(insert_data)

In [59]:
brandi_shop.index = range(len(brandi_shop))

In [159]:
brandi_shop.head()

,cs_url,shop_name,shop_tag
0,https://www.brandi.co.kr/shop/novemberfour/csc...,노벰버포,#20대중반#심플베이직#쇼핑몰
1,https://www.brandi.co.kr/shop/bbibbongunni/csc...,삐뽕언니,#20대후반#캐주얼#쇼핑몰
2,https://www.brandi.co.kr/shop/heylady/cscenter,헤이레이디,#20대중반#페미닌#쇼핑몰
3,https://www.brandi.co.kr/shop/elfya/cscenter,엘프야,#20대후반#페미닌#마켓
4,https://www.brandi.co.kr/shop/sappun/cscenter,사뿐,#20대중반#심플베이직#쇼핑몰


In [333]:
brandi_shop.to_csv('브랜디쇼핑몰정보.csv',index=False,encoding='cp949')

# selenium 크롤링

In [155]:
b_url = brandi_shop['cs_url'].iloc[86]

In [156]:
browser.get(b_url)

In [126]:
info0 = browser.page_source

In [127]:
info1 = BeautifulSoup(info0,'html.parser')

In [128]:
info2 = info1.find('body')

In [129]:
info3 = info2.find('div',{'class':'cs_container frame padv70'}).find('table').find('td').find('ul')

In [130]:
info4 = info3.find_all('li')

# 코드

In [ ]:
brandi_shop_info2= pd.DataFrame({'쇼핑몰':[],
                               '연락처':[],
                               '카톡':[],
                               '주소':[]})

In [267]:
#753부터 시작
for p in range(2191,len(brandi_shop)+1):
    b_url = brandi_shop['cs_url'].iloc[p]
    name = brandi_shop['shop_name'].iloc[p]
    browser.get(b_url)
    time.sleep(3.5)
    info0 = browser.page_source
    info1 = BeautifulSoup(info0,'html.parser')
    info2 = info1.find('body')
    info3 = info2.find('div',{'class':'cs_container frame padv70'}).find('table').find('td').find('ul')
    info4 = info3.find_all('li')
    
    
    for i in range(len(info4)):
        if '연락처' in str(info4[i]):
            call = info4[i].text
        elif '카톡' in str(info4[i]):
            kakao = info4[i].text
        elif '주소' in str(info4[i]):
            mail = info4[i].text
        else:
            pass

    try:
        if call:
            pass
    except:
        call=''
    try:
        if kakao:
            pass
    except:
        kakao=''
    try:
        if mail:
            pass
    except:
        mail=''

    insert_data = pd.DataFrame({'쇼핑몰':[name],
                               '연락처':[call],
                               '카톡':[kakao],
                               '주소':[mail]})

    brandi_shop_info1 = brandi_shop_info1.append(insert_data)
    brandi_shop_info1.index = range(len(brandi_shop_info1))
    del(call)
    del(kakao)
    del(mail)


IndexError: single positional indexer is out-of-bounds

In [168]:
brandi_shop_info

,쇼핑몰,연락처,카톡,주소
0,노벰버포,연락처 : 1577-3452,,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)영..."
1,삐뽕언니,연락처 : 070-8623-3086,카톡문의 : 삐뽕언니,주소 : 48311 ) 부산 수영구 남천동 24-21층 삐뽕언니
2,헤이레이디,연락처 : 070-4006-0292,,"주소 : 02577 ) 서울 동대문구 왕산로19길 61 (용두동, 용두빌딩)선농테라..."
3,엘프야,연락처 : 1577-3452,,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)6..."
4,사뿐,연락처 : 02-1544-2767,,주소 : 02724 ) 서울 성북구 삼양로 69 (길음동)CJ택배 길음래미 대리점 ...
5,업타운홀릭,연락처 : 070-7875-9411,,주소 : 61470 ) 광주 동구 대인동 28-2우체국_업타운홀릭 발송 및 반품 주소지
6,앤드모어,연락처 : 1577-3452,,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)영..."
7,로렌하이,연락처 : 1661-5203,카톡문의 : 로렌하이,주소 : 04794 ) 서울 성동구 성수동2가 289-8보문빌딩 6층
8,하늘하늘,연락처 : 02-1599-0410,,주소 : 05634 ) 서울 송파구 방이동 130-4CJ대한통운 신방이대리점 하늘하...
9,블랙업,연락처 : 15666813,카톡문의 : jsj3029,주소 : 03992 ) 서울 마포구 동교동 203-43지하1층


In [238]:
cod = []
for i in range(len(brandi_shop_info)):
    if '\u30fb' in brandi_shop_info['주소'].iloc[i]:
        cod.append(i)
    else:
        pass
    

In [245]:
test_data = pd.DataFrame({'a':['안녕','바보']})

In [248]:
test_data['a'].iloc[0] = ''

In [239]:
cod

[289]

In [256]:
'\u30fb' in brandi_shop_info['주소'].iloc[289]

False

In [254]:
'\u30fb' in re.sub('\u30fb',"",brandi_shop_info['주소'].iloc[289])

False

In [255]:
brandi_shop_info['주소'].iloc[289] = re.sub('\u30fb',"",brandi_shop_info['주소'].iloc[289])

In [257]:
brandi_shop_info.to_csv('브랜디쇼핑몰1.csv',index=False, encoding='cp949')

In [211]:
brandi_shop_info1 = brandi_shop_info1.append(brandi_shop_info2)

In [212]:
brandi_shop_info1.index = range(len(brandi_shop_info1))

In [ ]:
brandi_shop_info1.to_csv('브랜디쇼핑몰2.csv',index=False, encoding='cp949')

In [270]:
brandi_shop_info = brandi_shop_info.append(brandi_shop_info1)

In [271]:
brandi_shop_info.index = range(len(brandi_shop_info))

In [272]:
brandi_shop_info

,쇼핑몰,연락처,카톡,주소
0,노벰버포,연락처 : 1577-3452,,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)영..."
1,삐뽕언니,연락처 : 070-8623-3086,카톡문의 : 삐뽕언니,주소 : 48311 ) 부산 수영구 남천동 24-21층 삐뽕언니
2,헤이레이디,연락처 : 070-4006-0292,,"주소 : 02577 ) 서울 동대문구 왕산로19길 61 (용두동, 용두빌딩)선농테라..."
3,엘프야,연락처 : 1577-3452,,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)6..."
4,사뿐,연락처 : 02-1544-2767,,주소 : 02724 ) 서울 성북구 삼양로 69 (길음동)CJ택배 길음래미 대리점 ...
5,업타운홀릭,연락처 : 070-7875-9411,,주소 : 61470 ) 광주 동구 대인동 28-2우체국_업타운홀릭 발송 및 반품 주소지
6,앤드모어,연락처 : 1577-3452,,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)영..."
7,로렌하이,연락처 : 1661-5203,카톡문의 : 로렌하이,주소 : 04794 ) 서울 성동구 성수동2가 289-8보문빌딩 6층
8,하늘하늘,연락처 : 02-1599-0410,,주소 : 05634 ) 서울 송파구 방이동 130-4CJ대한통운 신방이대리점 하늘하...
9,블랙업,연락처 : 15666813,카톡문의 : jsj3029,주소 : 03992 ) 서울 마포구 동교동 203-43지하1층


In [273]:
brandi_shop_info.to_csv('브랜디쇼핑몰.csv',index=False, encoding='cp949')

# 쇼핑몰 중복제거

In [275]:
w_shop  = pd.read_csv('여_솔루션.csv',engine = 'python')

In [276]:
w_shop

,분류,쇼핑몰명,회사명,대표명,연락처,이메일,쇼핑몰URL,위치,solution
0,여성,육육걸즈,육육걸즈,박예나,1688-1360,dpskek@hanmail.net,http://www.66girls.co.kr/,전북 전주시,cafe24
1,여성,임블리,부건에프엔씨,박준성,1688-6745,help@bugunfnc.com,http://www.imvely.com,서울시 금천구,cafe24
2,여성,난닝구,엔 라인,이정민,1644-8883,help@naning9.com,http://www.naning9.com/,서울시 종로구,another solution
3,여성,스타일난다,난다,김소희,0502-707-8888,nanda@stylenanda.com,http://www.stylenanda.com,서울시 마포구,cafe24
4,여성,핫핑,에스에스엘케이에이치,김여진,1588-9843,hotpingna@naver.com,http://www.hotping.co.kr,부산시 해운대구,cafe24
5,여성,소녀나라,소녀나라,구길리,1599-8861,help@sonyunanra.com,http://www.sonyunara.com,서울시 구로구,another solution
6,여성,모코블링,모코블링,권아람,1644-1069,mocobling@naver.com,http://www.mocobling.com/,청주시 청원구,makeshop
7,여성,고고싱,고고싱,황은숙,1688-3073,hkm0977@nate.com,http://www.ggsing.com,전북 전주시,cafe24
8,여성,더데이즈,데이즈컴퍼니,노태호,1522-8987,dejou7@naver.com,http://www.thedaze.kr/,서울시 동대문구,cafe24
9,여성,러브러브미,스타일플럭스,이성일,1688-9095,leekry@nate.com,http://www.loveloveme.com/,서울시 종로구,cafe24


In [278]:
aa = w_shop['쇼핑몰명']

In [282]:
bb = brandi_shop_info['쇼핑몰']

In [290]:
cc = bb.tolist() + aa.tolist()

In [302]:
tes = pd.DataFrame({'쇼핑몰':aa.tolist()})

In [303]:
tes

,쇼핑몰
0,육육걸즈
1,임블리
2,난닝구
3,스타일난다
4,핫핑
5,소녀나라
6,모코블링
7,고고싱
8,더데이즈
9,러브러브미


In [304]:
tes[tes['쇼핑몰']=='임블리']

,쇼핑몰
1,임블리


In [306]:
tes2 =pd.DataFrame({'쇼핑몰':bb.tolist()})

In [316]:
tes2['쇼핑몰'].iloc[150] == tes['쇼핑몰'].iloc[1]

True

In [321]:
tes3=tes.append(tes2)

In [326]:
tes3[tes3['쇼핑몰']=='임블리']['쇼핑몰'].unique

<bound method Series.unique of 1      임블리
150    임블리
Name: 쇼핑몰, dtype: object>

In [329]:
shop_ls = tes3['쇼핑몰'].tolist()

In [330]:
len(shop_ls)

3155

In [332]:
len(set(shop_ls))

3030

# 주소 추출

In [3]:
brandi_shop_info = pd.read_csv('브랜디쇼핑몰url포함.csv',engine='python')

In [4]:
brandi_shop_info['주소'].iloc[2222].split()

['주소', ':', '11183', ')', '경기', '포천시', '소흘읍', '무봉리', '9-42번째', '컨테이너', '에이팩토리']

In [5]:
big = []
small= []
for i in range(len(brandi_shop_info)):
    try:
        city = brandi_shop_info['주소'].iloc[i].split()
        big.append(city[4]) 
        small.append(city[5])
    except:
        big.append('')
        small.append('')

In [6]:
len(small)

2225

In [350]:
len(big)

2225

In [7]:
brandi_shop_info['big'] = big

In [8]:
brandi_shop_info['small'] = small

In [357]:
len(brandi_shop_info[brandi_shop_info['small']=='강동구'])

22

In [10]:
brandi_shop_info[brandi_shop_info['big']=='서울']

,쇼핑몰,연락처,카톡,주소,url,big,small
0,노벰버포,연락처 : 1577-3452,NaN,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)영...",http://novemberfour.co.kr/shopinfo/company.html,서울,성동구
2,헤이레이디,연락처 : 070-4006-0292,NaN,"주소 : 02577 ) 서울 동대문구 왕산로19길 61 (용두동, 용두빌딩)선농테라...",http://heylady.co.kr/,서울,동대문구
3,엘프야,연락처 : 1577-3452,NaN,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)6...",https://www.fmkorea.com/index.php?document_srl...,서울,성동구
4,사뿐,연락처 : 02-1544-2767,NaN,주소 : 02724 ) 서울 성북구 삼양로 69 (길음동)CJ택배 길음래미 대리점 ...,http://www.sappun.co.kr/,서울,성북구
6,앤드모어,연락처 : 1577-3452,NaN,"주소 : 04778 ) 서울 성동구 뚝섬로1길 63 (성수동1가, 영창디지털타워)영...",http://vidfolio.kr/?author=52,서울,성동구
7,로렌하이,연락처 : 1661-5203,카톡문의 : 로렌하이,주소 : 04794 ) 서울 성동구 성수동2가 289-8보문빌딩 6층,/url?sa=f&rct=j&url=http://laurenhi.com/&q=&es...,서울,성동구
8,하늘하늘,연락처 : 02-1599-0410,NaN,주소 : 05634 ) 서울 송파구 방이동 130-4CJ대한통운 신방이대리점 하늘하...,http://www.hn-hn.co.kr/,서울,송파구
9,블랙업,연락처 : 15666813,카톡문의 : jsj3029,주소 : 03992 ) 서울 마포구 동교동 203-43지하1층,http://black-up.kr/,서울,마포구
12,원오브,연락처 : 0260320472,카톡문의 : 원오브,"주소 : 07631 ) 서울 강서구 공항대로 200 (마곡동, 마곡지웰타워)7층",http://oneof.co.kr/,서울,강서구
13,에이인,연락처 : 07050674534,카톡문의 : freshmoreain,"주소 : 04782 ) 서울 성동구 연무장길 53 (성수동2가, 삼영빌딩)2층 (주...",https://a-in.kr/,서울,성동구


In [11]:
brandi_shop_info[brandi_shop_info['small']=='강동구']

,쇼핑몰,연락처,카톡,주소,url,big,small
123,채버블,연락처 : 070-8158-8880,카톡문의 : 채버블,주소 : 05283 ) 서울 강동구 상일로5길 5 (상일동)2층,https://chaebubble.com/,서울,강동구
337,데이니즈,연락처 : 01094267093,NaN,주소 : 05384 ) 서울 강동구 성내동 39-81층 데이니즈,https://www.instagram.com/dayneeds_/,서울,강동구
439,에이치룸,연락처 : 01051556455,카톡문의 : hroom2888,"주소 : 05335 ) 서울 강동구 천호대로157길 14 (천호동, 나비빌딩)2층6...",https://www.brandi.co.kr/shop/hroom,서울,강동구
539,21워너비,연락처 : 01094157235,카톡문의 : 21wannabe,"주소 : 05230 ) 서울 강동구 양재대로 1706 (고덕동, 아남아파트)3-1201",http://21wannabe.com/,서울,강동구
587,블랙마돈나,연락처 : 070-7424-6004,NaN,"주소 : 05354 ) 서울 강동구 양재대로113길 51 (길동, 우성아트빌)4층1...",http://blackmadonna.co.kr/,서울,강동구
606,드레스날다,연락처 : 1668-1788,카톡문의 : dressnalda,주소 : 05408 ) 서울 강동구 양재대로 1307 (성내동)301호 드레스날다,http://dressnalda.com/,서울,강동구
779,고하트,연락처 : 01011111111,카톡문의 : nicebarbie,"주소 : 05335 ) 서울 강동구 천호대로157길 14 (천호동, 나비빌딩)2층 ...",https://www.instagram.com/gomjung_/,서울,강동구
919,몬지샵,연락처 : 01040849925,카톡문의 : 몬지,주소 : 05323 ) 서울 강동구 올림픽로88길 23 (천호동)cj대한통운 강동천...,http://minishop.gmarket.co.kr/monzi0606,서울,강동구
1077,디랩,연락처 : 1544-4822,카톡문의 : 디랩,주소 : 05253 ) 서울 강동구 암사길 33 (암사동)레더랩 1층,http://www.daddyslab.com/,서울,강동구
1231,하리피플,연락처 : 07040659033,NaN,주소 : 05300 ) 서울 강동구 길동 73bs빌딩 4층,http://haripeople.com/,서울,강동구
